<a href="https://colab.research.google.com/github/hallpaz/3dsystems21/blob/main/assignments/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5
# Part 1 - Read and elaborate

1.1 Read the paper [ShapeNet: An Information-Rich 3D Model Repository](https://arxiv.org/abs/1512.03012). What can you say about the importance and the challenges of building a large dataset for 3D machine learning?


*Use this space to write your thoughts*

# Part 2 - Learn

2.0 Run the following cells and read the explanations. It's an opportunity to get some familiarization with PyTorch3D. In "part 3" you are going to run some experiments.

In [ ]:
!pip install torch torchvision
import os
import sys
import torch
if torch.__version__=='1.6.0+cu101' and sys.platform.startswith('linux'):
    !pip install pytorch3d
else:
    need_pytorch3d=False
    try:
        import pytorch3d
    except ModuleNotFoundError:
        need_pytorch3d=True
    if need_pytorch3d:
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git'

In [ ]:
import numpy as np
import torch

from pytorch3d.datasets import (
    R2N2,
    ShapeNetCore,
    collate_batched_meshes,
    render_cubified_voxels,
)
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    TexturesVertex,
    look_at_view_transform,
)

from pytorch3d.structures import Meshes
from torch.utils.data import DataLoader

# add path for demo utils functions 
import sys
import os
sys.path.append(os.path.abspath(''))

In [ ]:
import matplotlib.pyplot as plt

If using **Google Colab**, fetch the utils file for plotting image grids and the ShapeNet JSON files to map corresponding ids and categories:

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/docs/tutorials/utils/plot_image_grid.py

!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/pytorch3d/datasets/shapenet/shapenet_synset_dict_v2.json
!cp /content/shapenet_synset_dict_v1.json  /usr/local/lib/python3.6/dist-packages/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json
!cp /content/shapenet_synset_dict_v2.json  /usr/local/lib/python3.6/dist-packages/pytorch3d/datasets/shapenet/shapenet_synset_dict_v2.json

from plot_image_grid import image_grid

OR if running locally uncomment and run the following cell:

In [ ]:
# from utils import image_grid

# 2.1. Load the datasets

We'll work on a small subset of the ShapeNetCore version 2 dataset. 

The whole dataset has more than 25GB and can be downloaded following the instructions here: https://www.shapenet.org/. Check the instructions page for the .zip file with our selected small subset of ShapeNet. If you are running on Google Colab, we suggest you upload the uncompressed folders to you Google Drive account and access its path here.

After downloading the data, modify `SHAPENET_PATH` below to you local path to the ShapeNetCore dataset folder. 

In [ ]:
# Setup
if torch.cuda.is_available():
 device = torch.device("cuda:0")
 torch.cuda.set_device(device)
else:
 device = torch.device("cpu")

SHAPENET_PATH = ""
shapenet_dataset = ShapeNetCore(SHAPENET_PATH,version=2)

We can retrieve a model by indexing into the loaded dataset. We can examine the category this model belongs to (in the form of a synset id, equivalent to wnid described in ImageNet's API: http://image-net.org/download-API), its model id, and its vertices and faces.

In [ ]:
shapenet_model = shapenet_dataset[1]
print("This model belongs to the category " + shapenet_model["synset_id"] + ".")
print("This model has model id " + shapenet_model["model_id"] + ".")
model_verts, model_faces = shapenet_model["verts"], shapenet_model["faces"]

We can use its vertices and faces to form a `Meshes` object which is a PyTorch3D datastructure for working with batched meshes.

In [ ]:
model_textures = TexturesVertex(verts_features=torch.ones_like(model_verts, device=device)[None])
shapenet_model_mesh = Meshes(
    verts=[model_verts.to(device)],   
    faces=[model_faces.to(device)],
    textures=model_textures
)

## 2.2. Render ShapeNetCore models with PyTorch3D's differentiable renderer

The `ShapeNetCore` dataloader has a customized `render` function that support rendering models by specifying their model ids, categories or indices using PyTorch3D's differentiable renderer implementation.

In [ ]:
# Rendering settings.
R, T = look_at_view_transform(1.0, 1.0, 90)
cameras = FoVPerspectiveCameras(R=R, T=T, device=device)
raster_settings = RasterizationSettings(image_size=512)
lights = PointLights(location=torch.tensor([0.0, 1.0, -2.0], device=device)[None],device=device)

First we will try to render three models by their model ids:

In [ ]:
images_by_model_ids = shapenet_dataset.render(
    model_ids=[
        "1e4df43ee2f2da6967f9cc18b363cf72",
        "6f6ed9e0d29b64e714be24585075d395",
        "da0179a5b68f13586a6a687121d74e50"
    ],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(images_by_model_ids.cpu().numpy(), rows=1, cols=3, rgb=True)

We can also render models by their indices. For instance, if we would like to render the three models at indices 93, 109 and 200:

In [ ]:
images_by_idxs = shapenet_dataset.render(
    idxs=[93, 109, 200],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(images_by_idxs.cpu().numpy(), rows=1, cols=3, rgb=True)

Alternatively, if we are not interested in any particular models but would like see random models from some specific categories, we can do that by specifying `categories` and `sample_nums`. For example, if we would like to render 2 models from the category "mailbox" and 3 models from the category "washer", we can do the following:

In [ ]:
images_by_categories = shapenet_dataset.render(
    categories=["mailbox", "washer"], #we also have the category "remote" in this subset of ShapeNet
    sample_nums=[2, 3],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(images_by_categories.cpu().numpy(), rows=1, cols=5, rgb=True)

If we are not interested in any particular categories and just would like to render some random models from the whole dataset, we can set the number of models to be rendered in `sample_nums` and not specify any `categories`:

In [ ]:
random_model_images = shapenet_dataset.render(
    sample_nums=[5],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(random_model_images.cpu().numpy(), rows=1, cols=5, rgb=True)

# Part 3 - Plotly Visualization

PyTorch3D has a differentiable renderer which allows us to generate images and take derivatives of the images with respect to the scene parameters. A differentiable renderer is very powerful, but it more computing intensive and it requires some setup. 

In this assignment, we only want to visualize the meshes, so we'll use Ploty for that. 

In [ ]:
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib

3.0. Show the number of instances in our subset of the ShapeNet dataset using the function `len` on `shapenet_dataset`

3.1. Create a Meshes object by taking it's vertices and faces from a shapenet model. Name your variable `myfirstmesh` so the code on the cell below display correctly.


In [ ]:
##############################################################################
# Code for 3.0-3.1.
##############################################################################

In [ ]:
# Render the plotly figure
fig = plot_scene({
    "subplot1": {
        "shapenet_mesh": myfirstmesh
    }
})
fig.show()

As You just saw, the function `plot_scene` takes a dictionary of subplots and returns a plotly figure. Each subplot is a dictionary of kind `name: mesh`. 

3.2. You can render textured models using Ploty. Create a Meshes object passing `verts`, `faces` and `textures` arguments. `textures` must be a `TexturesVertex` object. It's not an easy task to create a `TextureVertex` directly from the ShapeNet model, so for now we suggest you create a simple tensor like a noisy color or other kind of pattern you can control. 

3.3. Create another Meshes object from ShapeNet and show both models in the same subplot. 
  
  **Extra - E.1: you can create a batch of Meshes instead of two or more individual Meshes objects. To do so, note that the arguments to build a Meshes instance are, in fact, lists.**


3.4. Each mesh is represented in its own coordinate system. This way, when you put them in the same subplot, they will overlap. This time, create a new subplot and plot the meshes. 

  **Extra - E.2: if you created a batch of Meshes, use the function** `plot_batch_individually`.


3.5. Experiment changing the figure size using `fig.update_layout(height:int,width:int)` . Experiment adding the arguments below to the `plot_scene` or `plot_batch_individually` functions.
```
  (...)
  xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
  yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
  zaxis={"backgroundcolor":"rgb(200, 230, 200)"},
  axis_args=AxisArgs(showgrid=True)
  (...)
  ```
  **Extra - E.3: Create a `Meshes` instance by computing vertices and faces of a parametric model and visualize it.**

In [ ]:
##############################################################################
# Code for 3.2-3.5 (and extras).
##############################################################################